# Laboratorio 1 - Hill 
Por: Felipe Rojas Cendales  
UNAL | Introducción a la criptografía y la seguridad de la información

In [14]:
import numpy as np
import re

def makeKeyMatrix(key): 

  #Establish matrix and variables 
  keyMatrix = np.empty([2,2], dtype = int)
  row = 0
  col = 0

  #Add letters in key following order and if not repeated
  for number in key:
    number = int(number)
    keyMatrix[row][col] = number
    if col == 1: 
      row = row + 1 
      col = 0
    else: 
      col = col + 1

  return keyMatrix 

def lettersToNumbers(msg): 

  #Make msg even 
  if len(msg) % 2 != 0: 
    msg = msg + 'X'

  msgNumbers = []
  for letter in msg: 
    msgNumbers.append(ord(letter) - 65)
    
  return msgNumbers

def eea(a, b): 
  if b == 0: 
    return (a, 1, 0)
  q = int(a/b)
  (d, xP, yP) = eea(b, a % b)
  x = yP
  y = xP - (q * yP) 
  return (d, x, y)

def inverseMatrix(keyMatrix): 

  invKeyMatrix = np.empty([2,2], dtype = int)

  # Calculate det(keyMatrix)
  det = (keyMatrix[0][0] * keyMatrix[1][1]) - (keyMatrix[0][1] * keyMatrix[1][0])

  # Calculate Adj(keyMatrix)
  adj = np.empty([2,2], dtype = int)
  adj[0][0] = keyMatrix[1][1]
  adj[1][1] = keyMatrix[0][0]
  adj[0][1] = keyMatrix[0][1] * -1
  adj[1][0] = keyMatrix[1][0] * -1

  # Find inverse modular of det 

  greatestCommonDivisor, invDet, x = eea(det, 26)

  if greatestCommonDivisor != 1: 
    return 'Matrix not has inverse'

  # Calculate inverse of key

  invKeyMatrix = (invDet * adj) % 26

  return invKeyMatrix

def encrypt(keyMatrix, msg): 

  encryptedText = ''

  #Make msg even 
  if len(msg) % 2 != 0: 
    msg = msg + 'X'

  # Make tuples of letters from (0,1), make matrix multiplication and find letters 
  for (letterNumber1, letterNumber2) in zip(msg[0::2], msg[1::2]):
    encLetterNumber1 = ((letterNumber1 * keyMatrix[0][0]) + (letterNumber2 * keyMatrix[1][0])) % 26
    encLetterNumber2 = ((letterNumber1 * keyMatrix[0][1]) + (letterNumber2 * keyMatrix[1][1])) % 26

    encLetter1 = chr(encLetterNumber1 + 65)
    encLetter2 = chr(encLetterNumber2 + 65)

    encryptedText += encLetter1 + encLetter2 
    
  return encryptedText

def decrypt(keyMatrix, msg): 

  decryptedText = ''

  #Make msg even 
  if len(msg) % 2 != 0: 
    msg = msg + 'X'

  # Find matrix inverse
  keyMatrixInv = inverseMatrix(keyMatrix) 

  if type(keyMatrixInv) == str: 
    return keyMatrixInv

  # Make tuples of letters from (0,1), make matrix multiplication and find letters  
  for (letterNumber1, letterNumber2) in zip(msg[0::2], msg[1::2]):
    decLetterNumber1 = ((letterNumber1 * keyMatrixInv[0][0]) + (letterNumber2 * keyMatrixInv[1][0])) % 26
    decLetterNumber2 = ((letterNumber1 * keyMatrixInv[0][1]) + (letterNumber2 * keyMatrixInv[1][1])) % 26

    decLetter1 = chr(decLetterNumber1 + 65)
    decLetter2 = chr(decLetterNumber2 + 65)

    decryptedText += decLetter1 + decLetter2 
    
  return decryptedText

def hill():

  #Receive input, omit spaces, upper it, split by ; and validate
  key = input('Please, enter your key (the 4 key numbers must be separated with ;): ')
  key = key.strip()
  key = key.replace(' ','')
  keyArray = key.split(";")
  if len(keyArray) != 4: 
    return 'Key must have 4 numbers exactly'
  
  #Check key pattern as numbers
  pattern = re.compile('[0-9]*')
  for number in keyArray: 
    if pattern.fullmatch(number) == None: 
      return 'Key must be a valid number'

  #Generate Key matrix  
  keyMatrix = makeKeyMatrix(keyArray)

  #Receive message, omit spaces and upper it 
  msg = input('Please, enter your message: ')
  msg = msg.strip()
  msg = msg.replace(' ','')
  msg = msg.upper()

  #Check msg pattern
  pattern = re.compile('[A-Z]*')
  if pattern.fullmatch(msg) == None: 
    return 'Message must be valid without numbers and special characters'

  #Make msg letter to numbers
  msg = lettersToNumbers(msg)

  #Receive mode 1 to encrypt, 0 to decrypt
  mode = input('Please, enter the program mode (1 to Encrypt, 0 to Decrypt): ')
  mode = int(mode)
  if mode != 1 and mode != 0: 
    return('Invalid mode.') 

  #Encrypt/Decrypt, replace I for I/J and return
  if mode == 1:
    return print('Your message encrypted: ', encrypt(keyMatrix, msg))
  else: 
    return print('Your message decrypted: ', decrypt(keyMatrix, msg))
  
hill()

Please, enter your key (the 4 key numbers must be separated with ;): 0;0;0;0
Please, enter your message: dasd
Please, enter the program mode (1 to Encrypt, 0 to Decrypt): 0
Your message decrypted:  Matrix not has inverse
